In [1]:
# для примера использую библиотеку neuralprophet
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet

In [28]:
airports = pd.read_csv('airports.csv')
fligshts = pd.read_csv('flights.csv')

WARNING - (py.warnings._showwarnmsg) - /var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_85091/4155919845.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  fligshts = pd.read_csv('flights.csv')



In [29]:
#объединяю данные, что бы в данных было найменование аэропорта и код аэропорта
#датасет с цифровой расшифровкой аэропортов не нашел, здесь так же удаляю эти данные

fligshts = fligshts.merge(airports, how='right', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')

In [30]:
#добавляю колонку с датой в формате datetime64

fligshts['Date'] = fligshts[['YEAR','MONTH','DAY']].astype('str').agg('-'. join , axis= 1)
fligshts['Date'] = pd.to_datetime(fligshts['Date']).dt.date.astype('datetime64[ns]')

#удаляю колонки с датой, месяцем, годом

fligshts.drop(['YEAR', 'DAY', 'MONTH'], axis=1, inplace=True)


In [32]:
fligshts.head(2)

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,Date
0,4,EV,5103,N846AS,ABE,DTW,600,552.0,-8.0,12.0,...,NaN,NaN,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404,2015-01-01
1,4,EV,5579,N136EV,ABE,ATL,635,628.0,-7.0,13.0,...,NaN,NaN,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404,2015-01-01


In [33]:
#здесь формирую колонку PROBABILITY - если самолет опоздал, то значение 0
#прилетел раньше или в срок - значение 1

fligshts = fligshts[['Date', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']].reset_index()
fligshts['ARRIVAL_DELAY'] = fligshts['ARRIVAL_DELAY'].dropna().astype('int')
fligshts['PROBABILITY'] = np.where(fligshts['ARRIVAL_DELAY']>0, 0, 1)


In [34]:
fligshts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5332914 entries, 0 to 5332913
Data columns (total 6 columns):
 #   Column               Dtype         
---  ------               -----         
 0   index                int64         
 1   Date                 datetime64[ns]
 2   ORIGIN_AIRPORT       object        
 3   DESTINATION_AIRPORT  object        
 4   ARRIVAL_DELAY        float64       
 5   PROBABILITY          int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 244.1+ MB


In [35]:
# получаю список аэропортов вылета

ORIGIN_AIRPORT = list(fligshts.ORIGIN_AIRPORT.unique())
ORIGIN_AIRPORT

['ABE',
 'ABI',
 'ABQ',
 'ABR',
 'ABY',
 'ACK',
 'ACT',
 'ACV',
 'ACY',
 'ADK',
 'ADQ',
 'AEX',
 'AGS',
 'AKN',
 'ALB',
 'ALO',
 'AMA',
 'ANC',
 'APN',
 'ASE',
 'ATL',
 'ATW',
 'AUS',
 'AVL',
 'AVP',
 'AZO',
 'BDL',
 'BET',
 'BFL',
 'BGM',
 'BGR',
 'BHM',
 'BIL',
 'BIS',
 'BJI',
 'BLI',
 'BMI',
 'BNA',
 'BOI',
 'BOS',
 'BPT',
 'BQK',
 'BQN',
 'BRD',
 'BRO',
 'BRW',
 'BTM',
 'BTR',
 'BTV',
 'BUF',
 'BUR',
 'BWI',
 'BZN',
 'CAE',
 'CAK',
 'CDC',
 'CDV',
 'CEC',
 'CHA',
 'CHO',
 'CHS',
 'CID',
 'CIU',
 'CLD',
 'CLE',
 'CLL',
 'CLT',
 'CMH',
 'CMI',
 'CMX',
 'CNY',
 'COD',
 'COS',
 'COU',
 'CPR',
 'CRP',
 'CRW',
 'CSG',
 'CVG',
 'CWA',
 'DAB',
 'DAL',
 'DAY',
 'DBQ',
 'DCA',
 'DEN',
 'DFW',
 'DHN',
 'DIK',
 'DLG',
 'DLH',
 'DRO',
 'DSM',
 'DTW',
 'DVL',
 'EAU',
 'ECP',
 'EGE',
 'EKO',
 'ELM',
 'ELP',
 'ERI',
 'ESC',
 'EUG',
 'EVV',
 'EWN',
 'EWR',
 'EYW',
 'FAI',
 'FAR',
 'FAT',
 'FAY',
 'FCA',
 'FLG',
 'FLL',
 'FNT',
 'FSD',
 'FSM',
 'FWA',
 'GCC',
 'GCK',
 'GEG',
 'GFK',
 'GGG',
 'GJT',


In [ ]:
# и далее идея следующая - для каждого аэропорта вылета в цикле перебрать аэропорты прилета


for airport in ORIGIN_AIRPORT:
    df = fligshts.query(f'DESTINATION_AIRPORT == "{airport}"') #датафрейм с конкретным аэропортом прилета
    DESTINATION_AIRPORT = list(df.DESTINATION_AIRPORT.unique()) #список аэропортов прилета для конкретного аэропорта вылета
    for d_airport in DESTINATION_AIRPORT:
        A = df.query(f'DESTINATION_AIRPORT == "{d_airport}"') #фрейм с конкретным аэропортом прилета
        B = A.groupby('Date')['PROBABILITY'].mean().reset_index() #фрейм с датой и средним значением вероятности задержки в этот аэропорт по дням
        B.columns = ['ds', 'y'] #привожу колонки к наименованию, которые необходимы для модели
        m = NeuralProphet() #создаем экземпляр класса
        metrics = m.fit(B, freq="D") #обучаем модель
        
# Далее идея записать результат в pickle  - папки с наименованием аэропорта отлета, файлы с наименованием прилета(в соответствии с циклом)
# далее пользователь вводит наименование аэропорта отлета, и, например, дату, выгружаю модели из соответствующей папки 
# делаю предсказание, отвечаю на вопрос задания и сопутствующую визуализацию строю
        



In [42]:
metrics

,SmoothL1Loss,MAE,RMSE,RegLoss
0,1.786396,1.486886,1.685861,0.0
1,1.727579,1.447379,1.653334,0.0
2,1.662389,1.403771,1.602406,0.0
3,1.589291,1.354814,1.559216,0.0
4,1.502727,1.296815,1.499185,0.0
...,...,...,...,...
284,0.013615,0.086039,0.107258,0.0
285,0.013609,0.086027,0.105941,0.0
286,0.013609,0.086022,0.106358,0.0
287,0.013608,0.086017,0.105395,0.0


In [ ]:
pred = m.make_future_dataframe(B, periods=2)
forecast = m.predict(pred)